# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [4]:
weather_df=pd.read_csv('output/cities.csv')

weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,taolanaro,-54.8000,-68.3000,39.20,93,75,21.85,AR,1619652120
1,ushuaia,56.3491,78.3619,44.17,86,100,10.29,RU,1619652355
2,severnoye,71.2906,-156.7887,21.20,86,90,10.36,US,1619652356
3,barrow,-24.7911,-50.0119,53.91,85,15,3.29,BR,1619652356
4,castro,-33.5906,26.8910,66.27,79,98,7.18,ZA,1619652126
...,...,...,...,...,...,...,...,...,...
551,manadhoo,50.7990,-1.0913,46.99,81,100,20.24,GB,1619652596
552,karpogory,62.2167,135.8000,34.43,99,100,0.45,RU,1619652740
553,cerchezu,-20.2208,-70.1431,66.20,68,0,11.50,CL,1619652741
554,port blair,-2.9900,-65.1083,78.80,89,40,2.30,BR,1619652741


### Humidity Heatmap

Configure gmaps.Use the Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.Drop any rows will null values.

## Hotel Map
Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure